In [3]:
# All Imports
import pyttsx3 as tts
import speech_recognition as sr
import PyDictionary as pyd
from email.message import EmailMessage
from newsapi import NewsApiClient
import datetime
import wikipedia
import webbrowser
import requests
import os
import json
import smtplib
import ssl
import randfacts
import wolframalpha
import re
import pyjokes
import dadjokes
import random
import subprocess
import winshell
import time
import socket,colorama


In [4]:

def isConnected():
    try:
        # connect to the host -- tells us if the host is actually
        # reachable
        sock = socket.create_connection(("www.google.com", 80))
        if sock is not None:
            sock.close
        return True
    except OSError:
        pass
    return False
isConnected()

True

In [30]:
# Global Variables
from operator import contains
from pprint import pprint


user_profile = json.load(open('user_profile.json'))["user"]
contacts = json.load(open('user_profile.json'))["contacts"]
settings = json.load(open("va_settings.json"))
stored_Websites = settings["websites"]
stored_API = json.load(open("api.json"))


voices = json.load(open("voices.json"))
default_voice = getVoice()
# storeWeather()

commands = json.load(open("commands.json"))
for k, v in commands.items():
    commands[k] = list(map(lambda x: x.lower().strip(), v))


TypeError: 'module' object is not callable

In [6]:
# Setting up the engine
engine = tts.init()
engine.setProperty('rate', 200)
engine.setProperty('voice', default_voice['id'])


In [7]:
def speak(text: str):
    """Inputted text is spoken"""
    text = " ".join([w.capitalize() for w in text.split(" ")]).strip()
    engine.say(text)
    print(default_voice["callName"].capitalize()+": " + text)
    engine.runAndWait()


def updateVoices():
    """Updates the voices available"""
    x = {}
    voices = engine.getProperty('voices')
    for i, v in enumerate(voices):
        x.setdefault(i, {})
        x[i]["name"] = v.name
        x[i]["id"] = v.id
        x[i]["default"] = "false"
        x[i]["index"] = i
        x[i]["callName"] = v.name.split(" ")[1]
    x[1]["default"] = "true"
    json.dump(x, open("data/voices.json", "w"), indent=4)


def changeVoice(default_voice):
    """Changes the voice of the assistant"""
    curr_index = int(default_voice['index'])
    new_index = "0" if curr_index >= len(voices)-1 else str(curr_index+1)
    engine.setProperty('voice', voices[new_index]['id'])
    return voices[new_index]


def getVoice():
    for k in voices.keys():
        if voices[k]['default'] == "true":
            return voices[k]


In [8]:
def wishMe():
    """Greet the User"""
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning! ")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon! ")
    else:
        speak("Good Evening!")

    speak("I am "+default_voice['callName'] +
          ". Please tell me how may I help you")


In [9]:
def acceptCommand():
    """Accepts the user's command via the microphone and returns the string output"""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.energy_threshold = 10000
        print("Listening...")
        # seconds of non-speaking audio before a phrase is considered complete
        r.adjust_for_ambient_noise(source, duration=0.5)
        r.pause_threshold = 1
        audio = r.listen(source)
        try:
            query = r.recognize_google(audio, language=settings["language"])
            print(user_profile["name"]+": ", query, end="\n")
        except Exception as e:
            speak("Unable to recognize your voice. Please say that again...")
            return "None"
    return query.lower()


In [38]:
import pprint


def sendMail(query):
    # BUG Receiver Error
    """Send a Mail: import ssl,smtplib,email.message.EmailMessage"""
    sender = user_profile["email"]
    password = user_profile["password"]

    receiver = stripCommand(query, "email")
    if receiver not in contacts or receiver == "":
        receiver = str(input("Receiver ?"))
    em = EmailMessage()
    em['From'] = sender
    em['To'] = receiver
    em['Subject'] = str(input("Subject:"))
    em.set_content(str(input("Content: ")))

    context = ssl.create_default_context()
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
            smtp.login(sender, password)
            smtp.sendmail(sender, receiver, em.as_string())
            speak("Mail Sended")
    except Exception as e:
        speak("Unable to Send the Mail !!")


In [11]:
def searchWikipedia(query):
    """Searches Wikipedia for the user's query"""
    words_in_query = query.split(" ")
    for command in commands['search']:
        for words in command.strip().split():
            if words.lower() in words_in_query:
                words_in_query.remove(words.lower())
    print(" ".join(words_in_query))
    # q = wikipedia.search(query)[0]
    # results = wikipedia.summary(q, sentences=1, auto_suggest=False)
    # results = wikipedia.summary(query, sentences=2)
    # print(results)
    # speak(q+", According to Wikipedia")
    # speak(results)
searchWikipedia("I want to search neuron".lower())


neuron


In [12]:
def getFacts():
    """Returns a random fact"""
    speak("Here is a random fact")
    speak(randfacts.get_fact())


def getJokes():
    """Returns a random joke"""
    speak("Here is a random joke")
    joking = dadjokes.Dadjoke()
    speak(joking.joke)


In [13]:
def playMusic(query):
    """Plays music from the user's directory"""
    music_dir = json.load(open("data/user_profile.json"))["music-dir"]
    songs = [s.lower() for s in os.listdir(music_dir)]
    print(songs)

    for c in commands['music']:
        if query.__contains__(c):
            print(query)
            break

    if query+".mp3" in songs:
        os.startfile(music_dir+"\\" + query + ".mp3")
    elif query == '':
        os.startfile(music_dir+"\\"+random.choice(songs))
    else:
        speak("I don't know that song")


In [14]:
def playYouTube(query):  # TODO Incomplete
    """Plays a YouTube video from the user's query"""
    for c in commands['youtube']:
        if c in query:
            query = query.replace(c, '')
            break
    query = query.strip().replace(' ', '+')
    try:
        webbrowser.open(
            "https://www.youtube.com/results?search_query=" + query)
    except Exception as e:
        speak("Unable to play youtube, please try again.")


In [15]:
def solveProblems():
    """Perform basic problems solving"""

    client = wolframalpha.Client(stored_API["wolframalpha"]["client"])
    speak("What do you want to ask ?")
    solve_query = acceptCommand()
    response = client.query(solve_query)
    try:
        answer = next(response.results).text
        speak(answer)
    except Exception as e:
        speak("Unable to solve, please try again")


In [16]:
def useDictionary(query):
    """Get Meaning"""
    dictionary = pyd.PyDictionary()
    for c in commands["dictionary"]:
        if c in query:
            query = query.replace(c, "")
            break
    try:
        result = dictionary.meaning(query)
        print(result)
        # speak("According to dictionary,")
        # for k, v in result.items():
        #     #TODO Make it Voice
        #     print("> "+k+": "+"\n\t>".join(v))
    except Exception as e:
        print(e)
        speak("I don't know that")


In [17]:
def openWebsite(query):
    """Opens a website from the user's query"""
    for c in commands['website']:
        if c in query:
            website = query.replace(c, "").strip()
            break
    if stored_Websites.__contains__(website):
        webbrowser.open(stored_Websites[website])
    else:
        speak("I don't know that website.")


In [18]:
def getNews():
    client = NewsApiClient(api_key=stored_API['news']['client'])
    top_art = client.get_top_headlines(
        country="in", language=settings["language"])['articles']
    title = [t['title'].split("-") for t in top_art][0:3]
    speak("These are top three headlines: ")
    engine.setProperty('rate', 200)
    for t in title:
        speak(t[0]+"from"+t[1])
    engine.setProperty('rate', 150)


In [19]:
from turtle import fd


def getTime():
    speak("The Time is "+datetime.datetime.now().strftime("%H:%M"))


def getDate():
    speak("Today's Date is "+str(datetime.datetime.now().date()))


def lockWin():
    speak("Locking the Device")
    subprocess.call('rundll32.exe user32.dll, LockWorkStation')


def emptyBin():
    winshell.recycle_bin().empty(confirm=False, show_progress=False, sound=True)
    speak("Emptied Recycle Bin")


def goToSleep():
    speak("For how much time you want to stop from listening commands ?")
    time.sleep(int(acceptCommand()))


def addTask():
    # speak("What should I write ?")
    # TODO Do it JSON
    note = "Download"
    note = "\n- [ ] " + \
        "Testing ("+datetime.datetime.now().strftime("%H:%M:%S")+")"

    file = open("notes.md", "a")
    file.write(note)


def clearTask():
    # TODO Clear Task
    pass


def showTask():
    speak("Showing Task List")
    file = open("notes.md")
    print(file.read())


In [20]:
from datetime import date
from pprint import pprint
import weakref
import requests


def storeWeather():
    open_weather = stored_API["openweather"]
    stored_date = json.load(open("weather.json"))['date']
    today = str(datetime.datetime.now().date())
    if stored_date == today:
        return
    else:
        print("Storing")
        location = settings['city'].strip()

        weather_data = requests.get(
            open_weather['url']+location+"&appid="+open_weather['client']).json()
        dump_weather = {}
        dump_weather.setdefault('date', str(datetime.datetime.now().date()))
        dump_weather.setdefault('weather', weather_data["main"])
        dump_weather.setdefault(
            'description', weather_data["weather"][0]["main"])
        dump_weather.setdefault('wind-speed', weather_data["wind"]["speed"])
        dump_weather.setdefault('wind-angle', weather_data["wind"]["deg"])
        dump_weather.setdefault('city', weather_data["name"])
        json.dump(dump_weather, open("weather.json", "w"), indent=4)


def retrieveWeather(query):
    weatherData = json.load(open("weather.json"))
    print(weatherData)
    today = str(datetime.datetime.now().date())
    if weatherData["date"] == today:
        print("Showing")
        location = settings['city'].strip()
        qk = parseQuery(query)
        for c in commands['weather']:
            if c in query:
                query = query.replace(c, '').strip()
        speak("Today's Forecast For "+location.capitalize()+" is: ")
        speak("Min Temperature: " +
              str(weatherData['weather']['temp_min'])+" Kelvin")
        speak("Max Temperature: " +
              str(weatherData['weather']['temp_max'])+" Kelvin")
        speak("Humidity: "+str(weatherData['weather']['humidity'])+"%")
        speak("Description:"+str(weatherData['description']))
        if query == "verbose":
            speak("Wind Speed: "+str(weatherData["wind-speed"]))
            speak("Wind Angle: "+str(weatherData["wind-angle"]))
            speak("Atmospheric Pressure: " +
                  str(weatherData["weather"]["pressure"])+" millibar")
    else:
        speak("Unable to Forecast,Cause of Rain & Showers I Guess.")


storeWeather()


In [40]:
def parseQuery(query):
    query_key = ''
    for k, c in commands.items():
        for v in c:
            if re.findall(v, query):
                print(re.findall(v, query))
                query_key = k
                return query_key


def stripCommand(query: str, query_key: str):
    words_in_query = query.split(" ")
    for command in commands[query_key]:
        for words in command.strip().split():
            if words.lower() in words_in_query:
                words_in_query.remove(words.lower())
    return " ".join(words_in_query).strip()


for k,v in commands.items():
    for c in v:
        print(stripCommand(c,k))